In [1]:
import numpy as np
import sys
sys.path.append('.')
from step15_16_main import *

## 16. 복잡한 계산 그래프 (구현 편)
### 16.1. 세대 추가

```python
class Variable:
    def __init__(self, data):
        ...
        self.generation = 0

    def set_creator(self, func):
        ...
        self.generation = func.generation + 1
```
함수의 세대는 입력값의 세대 중에서 가장 큰 값을 밭아 오는 것으로 한다.
```python
class Function:
    def __call__(self, *inputs):
        ...
        self.generation = max([x.generation for x in inputs])

```

### 16.2-3. 세대 순으로 꺼내기 구현
Var class 에서 함수 세대별로 정리 및 순차적인 grad 값 계산 및 추출 구현
```python
class Variable:
    ...
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()  # 집합 : 중복 피하기 가능

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx

                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)
```

In [2]:
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)


32.0
64.0


In [6]:
print(x.generation)
print(a.generation)
print(y.generation)


0
1
3
